In [2]:
import numpy as np
import tensorflow as tf

# train test split

In [3]:
from keras.utils import image_dataset_from_directory

In [4]:
ds_train = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 5000 files belonging to 50 classes.
Using 4000 files for training.
Found 5000 files belonging to 50 classes.
Using 1000 files for validation.


In [5]:
# Obtenir le nombre de classes à partir de ds
num_classes = len(ds_train.class_names)

print('Nombre de classes :', num_classes)

Nombre de classes : 50


In [6]:
import tensorflow as tf

# Convertir les étiquettes cibles en vecteurs one-hot
ds_train = ds_train.map(lambda x, y: (x, tf.one_hot(y, num_classes)))
ds_test = ds_test.map(lambda x, y: (x, tf.one_hot(y, num_classes)))

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.config.list_physical_devices()

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(num_classes, activation='relu')(x_2)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             TensorBoard(log_dir='./logs/relu', histogram_freq=0, write_graph=True, write_images=True)]

# Entraîner le modèle sur votre dataset avec le callback personnalisé

model.fit(ds_train, epochs=10, callbacks=callbacks, validation_data=ds_test)

Epoch 1/10


2024-07-02 15:06:29.922446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


63/63 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.0749 - loss: 3.6987 - val_accuracy: 0.3320 - val_loss: 2.6194
Epoch 2/10
40/63 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.3880 - loss: 2.3637

In [ ]:
model.save_weights('current_model.weights.h5')

In [ ]:
import json

# Stocker les noms de classes dans une liste
class_names = ds_train.class_names

# Stocker la liste dans un fichier JSON
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(ds_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(ds_test.labels, axis=1)
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
precision_by_class = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for i, class_name in enumerate(class_names):
    print(f"Precision for {class_name}: {precision_by_class[i]:.4f}")